In [2]:
import os
from glob import glob
import sys
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

sys.path.insert(0, '../')
from dataset import get_dataloader

In [3]:
TaskMetaInfo.Encoder

{'mask': {'incorrect': 0, 'wear': 1, 'not_wear': 2},
 'gender': {'male': 0, 'female': 1}}

In [8]:
RANDOM_SEED = 42
TRAIN_RAW = '../input/data/train/images'

In [9]:
data = pd.read_csv('../input/data/train/train.csv')
data.head(3)

,id,gender,race,age,path
0,000001,female,Asian,45,000001_female_Asian_45
1,000002,female,Asian,52,000002_female_Asian_52
2,000004,male,Asian,54,000004_male_Asian_54


In [10]:
mask_states = ['incorrect_mask', 'mask1', 'mask2', 'mask3', 'mask4', 'mask5', 'normal']

def attach_img_path(x):    
    output = [os.path.join(x, class_) for class_ in mask_states]
    return output

def get_image_paths(x):
    name = os.path.basename(x)
    output = pd.DataFrame(np.array(attach_img_path(x)), columns=['image_path'])
    output['mask_detail'] = [m for m in mask_states]
    output['path'] = name
    return output

In [11]:
%%time
path_list = data['path'].tolist()
image_paths_raw = list(map(lambda x: get_image_paths(os.path.join(TRAIN_RAW, x)), path_list))
image_paths = pd.concat(image_paths_raw, ignore_index=True, axis=0)

data = data.merge(image_paths, how='left', on='path')
data.head(3)


CPU times: user 3.15 s, sys: 12 ms, total: 3.16 s
Wall time: 3.16 s


,id,gender,race,age,path,image_path,mask_detail
0,000001,female,Asian,45,000001_female_Asian_45,../input/data/train/images/000001_female_Asian...,incorrect_mask
1,000001,female,Asian,45,000001_female_Asian_45,../input/data/train/images/000001_female_Asian...,mask1
2,000001,female,Asian,45,000001_female_Asian_45,../input/data/train/images/000001_female_Asian...,mask2


In [18]:
def get_mask_class(x):
    if x == 'incorrect_mask':
        class_ = 'incorrect'
    elif x == 'normal':
        class_ = 'not_wear'
    else:
        class_ = 'wear'
    return class_


data['mask'] = data['mask_detail'].apply(lambda x: get_mask_class(x))
col_order = ['id', 'gender', 'age', 'mask', 'mask_detail', 'path', 'image_path']
data = data[col_order]
data.head(3)

,id,gender,age,mask,mask_detail,path,image_path
0,000001,female,45,incorrect,incorrect_mask,000001_female_Asian_45,../input/data/train/images/000001_female_Asian...
1,000001,female,45,wear,mask1,000001_female_Asian_45,../input/data/train/images/000001_female_Asian...
2,000001,female,45,wear,mask2,000001_female_Asian_45,../input/data/train/images/000001_female_Asian...


In [20]:
def tag_extension(img_path):
    for ex in ['.jpg', '.png', '.jpeg']:
        path = img_path + ex
        if os.path.isfile(path):
            break
    return path
data['image_path'] = data['image_path'].apply(lambda x: tag_extension(x))

In [31]:
img_abspath_list = data['image_path'].tolist()

In [36]:
encoder = TaskMetaInfo.Encoder

In [37]:
encoder

{'mask': {'incorrect': 0, 'wear': 1, 'not_wear': 2},
 'gender': {'male': 0, 'female': 1}}

In [34]:
label_meta = {}

In [38]:
meta = {}
sample = img_abspath_list[0]

# mask
def get_mask_cat(img_path):
    if 'incorrect' in img_path:
        return 'incorrect'
    elif 'normal' in img_path:
        return 'not_wear'
    else:
        return 'wear'

def get_mask_cat(img_path):
    if 'incorrect' in img_path:
        return 'incorrect'
    elif 'normal' in img_path:
        return 'not_wear'
    else:
        return 'wear'

'../input/data/train/images/000001_female_Asian_45/incorrect_mask.jpg'

In [17]:
TRAIN = '../preprocessed/train'
loader = get_dataloader('train', TRAIN, 'base', 32)

In [21]:
for imgs in loader:
    break

In [27]:
imgs, labels = cutmix(imgs, .5)

In [32]:
labels[2]

0.07352423161483547